In [ ]:
import pandas as pd
data = pd.read_csv("/Volumes/T5_SSD/DeepLearning/ok-cupid/user_data_public.csv")
#Convert columns to categorical in order to save 90% of memory
#print data.memory_usage()
for h in list(data):
    if h[0] == 'q':
        data[h] = data[h].astype('category')
#print data.memory_usage()
data.to_pickle("./unprocessed.bz2")

--------------------------------

In [1]:
import pandas as pd
import numpy as np
orig_data = pd.read_pickle("./unprocessed.bz2")
orig_data.drop(['gender2', 'gender2_num','CA_items','d_gender'], axis=1, inplace=True ) #Duplicates, d_gender~=gender
orig_data = orig_data.replace(np.nan, 'Nan', regex=True)

In [8]:
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import pandas as pd

question_headers = []
non_question_headers = []
for h in list(orig_data):
    if h[0] == 'q':
        question_headers.append(h)
    elif h[0] != 'p':
        non_question_headers.append(h)

print 'non_question_headers', non_question_headers, len(non_question_headers)
non_question_headers.remove('d_orientation')
#orig_data = orig_data[non_question_headers+question_headers[:5]].iloc[:200].copy()

non_question_data_one_hot = pd.DataFrame()
data = orig_data[non_question_headers].copy()
data['CA'] = data['CA'].replace('Nan', 0.0, regex=True)
data['CA'] = data['CA'].astype(np.float64)

#Normalize ages
for col in ['lf_max_age', 'lf_min_age', 'd_age']:
    data[col] = data[col].replace(to_replace='Nan',value=0).apply(lambda x: x/100)

#remove trialing 'For  '
data['lf_for'] = data['lf_for'].map(lambda x: str(x).lstrip('For'))
data['d_languages'] = data['d_languages'].replace(regex=True,to_replace=r'\(.*\)',value=r'')
#convert multi value columns into lists
multi_value = ['d_languages','lf_for','d_ethnicity']
for col in multi_value:
    data[col] = data[col].replace(regex=True,to_replace=r'\s+',value=r'')
    data[col] = data[col].str.split(",")

#keep top n languages
top_languages_len = 25
top_languages = set(data['d_languages'].apply(pd.Series).stack().value_counts().index.tolist()[:top_languages_len])
import math
data['d_languages'] = data['d_languages'].apply(lambda x: list(set(x) & top_languages))

#convert multi_value columns to one hot 
mlb = MultiLabelBinarizer()
for col in multi_value:
    column_one_hot_encoded = pd.DataFrame(mlb.fit_transform(data[col]),columns=mlb.classes_,index=data.index)
    non_question_data_one_hot = pd.concat([non_question_data_one_hot,column_one_hot_encoded], axis=1)
    data.drop(col, axis=1, inplace=True)

#append all the remanining data
for col in list(data):
    if np.issubdtype(data[col].dtype, np.number):
        print data[col].dtypes, col 
        non_question_data_one_hot = pd.concat([non_question_data_one_hot, data[col]], axis=1)    
    else:
        print data[col].dtypes, col 
        non_question_data_one_hot = pd.concat([non_question_data_one_hot, pd.get_dummies(data[col])], axis=1)

non_question_headers ['d_astrology_seriosity', 'd_education_phase', 'd_income', 'd_religion_type', 'd_drugs', 'lf_want', 'd_smokes', 'd_astrology_sign', 'd_country', 'd_drinks', 'd_age', 'd_ethnicity', 'd_offspring_current', 'd_bodytype', 'lf_max_age', 'lf_for', 'd_job', 'd_orientation', 'd_languages', 'd_relationship', 'd_offspring_desires', 'lf_location', 'lf_min_age', 'd_education_type', 'lf_single', 'd_religion_seriosity', 'CA', 'gender_orientation', 'gender', 'race'] 30
object d_astrology_seriosity
object d_education_phase
object d_income
object d_religion_type
object d_drugs
object lf_want
object d_smokes
object d_astrology_sign
object d_country
object d_drinks
float64 d_age
object d_offspring_current
object d_bodytype
float64 lf_max_age
object d_job
object d_relationship
object d_offspring_desires
object lf_location
float64 lf_min_age
object d_education_type
object lf_single
object d_religion_seriosity
float64 CA
object gender_orientation
object gender
object race


In [9]:
#https://www.pyimagesearch.com/2019/02/04/keras-multiple-inputs-and-mixed-data/

# non_question_data_one_hot.to_pickle("./non_question_data_one_hot.bz2") #input

selected_question_headers = question_headers #["q9688","q12719"]
selected_question_headers = ["q9688"]
question_data_one_hot = pd.get_dummies(orig_data[selected_question_headers])
# question_data_one_hot.to_pickle("./question_data_one_hot.bz2") #out labels

pd.concat([question_data_one_hot, non_question_data_one_hot], axis=1).to_pickle("./one_hot.bz2")

In [5]:
#DISPLAY UNIQUE VALS FOR ALL COLUMNS
# for h in non_question_headers:
#     print data[h].apply(pd.Series).stack().value_counts()[:20], '\n\n'

# for i in [100,1000,256,365,512,2000]:
#     print 'Get unique vals for column ', data.iloc[:,i].unique()

#Print column types
# print out_data.dtypes 

#Find columns with most NA
# orig_data = pd.read_pickle("./unprocessed.bz2")
# orig_data[question_headers].isna().sum().sort_values()


# print data.iloc[:5000]['gender_orientation'].apply(pd.Series).stack().value_counts()[:20], '\n\n'

# print  non_question_data_one_hot.shape, list(non_question_data_one_hot)
# print 
print question_data_one_hot.shape, list(question_data_one_hot)


(68371, 9877) ['q2_Nan', 'q2_more cool than pathetic', 'q2_more pathetic than cool', 'q11_Aroused', 'q11_Horrified', 'q11_Indifferent', 'q11_Nan', 'q11_Nostalgic', 'q12_Nan', 'q12_No', 'q12_Yes', 'q13_Nan', 'q13_No', 'q13_Yes', 'q14_Nan', 'q14_No', 'q14_Yes', 'q16_Nan', 'q16_No', 'q16_Yes', 'q17_Nan', 'q17_No', 'q17_Yes', 'q18_Nan', 'q18_No', 'q18_Yes', 'q20_Nan', 'q20_No', 'q20_Yes', 'q22_Nan', 'q22_No', 'q22_Yes', 'q24_Nan', 'q24_No', 'q24_Yes', 'q25_Nan', 'q25_No', 'q25_Yes', 'q26_Nan', 'q26_No', 'q26_Yes', 'q28_Nan', 'q28_No', 'q28_Yes', 'q29_Nan', 'q29_avoid bondage all together', 'q29_be tied up during sex', 'q29_do the tying', 'q30_Nan', 'q30_No', 'q30_Yes, lots and lots', 'q30_Yes, some', 'q32_I have little or no interest.', 'q32_I have tried it.', 'q32_It seriously interests me.', 'q32_Nan', 'q33_Nan', 'q33_No', 'q33_Yes', 'q35_Love', 'q35_Nan', 'q35_Sex', 'q36_Nan', 'q36_No', 'q36_Yes', 'q37_18 or Over', 'q37_I never have!', 'q37_Nan', 'q37_Under 18', 'q38_Nan', 'q38_No', 'q3